In [8]:
!pip install pandas openpyxl

In [10]:
import json
import pandas as pd

legislativas = pd.read_excel("dados/legislativas.xlsx",header=[0,1],sheet_name="Quadro")

In [29]:
display(legislativas)

Territórios                         Total                    \
    Âmbito Geográfico               Região     1975     1976     1979   
0           NUTS 2024             Portugal  6220784  6583311  6898589   
1              NUTS I           Continente  5876448  6171428  6450886   
2             NUTS II                Norte  1928866  2039847  2169661   
3            NUTS III           Alto Minho   157544   164640   172773   
4           Município    Arcos de Valdevez    20217    20825    20745   
..                ...                  ...      ...      ...      ...   
352         Município              Santana     6518     6760     6727   
353         Município          São Vicente     4748     4848     4946   
354          NUTS III  Ilha de Porto Santo     2369     2479     2537   
355         Município          Porto Santo     2369     2479     2537   
356            NUTS I       Ignorado/Outro    43086   105709   141437   

                                                  ... Votantes           \
        1980     1983     1985     1987     1991  ...     1991     1995   
0    7108987  7344032  7812770  7928687  8509911  ...  5735434  5900277   
1    6613766  6836119  7273779  7387809  7947700  ...  5444596  5614959   
2    2239377  2345399  2531326  2593934  2838675  ...  1985542  2074094   
3     178153   184167   196172   199593   214800  ...   139696   145068   
4      21559    22136    23618    23959    25374  ...    13647    13911   
..       ...      ...      ...      ...      ...  ...      ...      ...   
352     6891     7424     7505     8197     8226  ...     5270     5322   
353     5011     5236     5601     5623     5918  ...     3319     3444   
354     2638     2759     2990     3058     3424  ...     2551     2786   
355     2638     2759     2990     3058     3424  ...     2551     2786   
356   183744   184683   191266   187538   187430  ...    61102    45852   

                                                                             
        1999     2002     2005     2009     2011     2015     2019     2022  
0    5406946  5473631  5750354  5683967  5588594  5408805  5251064  5563497  
1    5148560  5218827  5481867  5425800  5326185  5161544  4879426  5178510  
2    1940375  1984261  2088998  2112227  2050289  1971899  1902891  2001871  
3     137179   139237   142197   141482   134902   128488   121913   127040  
4      13367    13343    12951    12519    11947    11926    11122    11262  
..       ...      ...      ...      ...      ...      ...      ...      ...  
352     4864     4977     5315     4899     4685     4015     3956     3625  
353     3210     3324     3553     3463     3257     2729     2851     2678  
354     2548     2464     2751     2886     2874     2546     2723     2792  
355     2548     2464     2751     2886     2874     2546     2723     2792  
356    43040    39707    36714    25472    33059    28354   158252   173792  

[357 rows x 36 columns]

In [ ]:

def eleitoresPorto() -> int:
    
    region_name = 'Área Metropolitana do Porto'
    interest_row = legislativas[local_legislativas[('Territórios', 'Região')] == region_name]
    return interest_row['Total'].max().idxmax()

eleitoresPorto()

In [ ]:
# get the first row of the dataframe
first_row = legislativas["Total"].iloc[0]

# Get the first element from the first row if it is a Series
first_element = first_row.iloc[0] if hasattr(first_row, 'iloc') else "The row is not a Series with multiple elements."
first_element

len(legislativas["Total"].columns)

In [ ]:


def taxaAbstencao() -> list[tuple[int,float]]:

    porcentagem_de_abstencao = []
    abstencao = []

    total = legislativas["Total"].iloc[0]
    eleitores = legislativas["Votantes"].iloc[0]
    porcentagem_de_abstencao.append(((total - eleitores) / total) * 100)

    for index in range(len(legislativas["Total"].columns)):
        abstencao.append((legislativas["Total"].columns[index], porcentagem_de_abstencao[0].iloc[index]))

    return abstencao

taxaAbstencao()

In [ ]:

def perdaGrandesMunicipios() -> dict[str,int]:
    # Por cada município com pelo menos 10000 votantes num dos anos, qual o ano em que mais perderam votantes em relação às eleições anteriores? Complete a definição da função perdaGrandesMunicipios, que retorna um dicionário { municipio : ano }.


    municipios_validos = {}

    # conseguir o a serie pandas com todos os municipios

    location_type = 'Município'
    municipal_rows = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]

    # primeiro achamos todos os municipios com pelo menos 10000 votantes num dos anos
    for row_index, row_serie in municipal_rows["Votantes"].iterrows():
        for year_index in range(len(municipal_rows["Votantes"].columns)):
            if row_serie.iloc[year_index] >= 10000:
                municipios_validos[legislativas['Territórios']['Região'].iloc[row_index]] = ""
                break
    
    # map the list index to the year
    years2index = {index: year for index, year in enumerate(municipal_rows["Votantes"].columns)}

    # agora vamos achar o ano em que mais perderam votantes em relação às eleições anteriores
    for municipio in municipios_validos:

        # find the row index of the municipio
        municipio_row_index = municipal_rows[municipal_rows['Territórios']['Região'] == municipio].index[0]
        # get the pandas series of the municipio
        municipio_data = legislativas["Votantes"].iloc[municipio_row_index]
        # transform the series into a list
        municipio_data_list_raw = municipio_data.tolist()
        # make a copy of the list
        municipio_data_list_shifted = municipio_data_list_raw.copy()
        # shift the list to the right
        municipio_data_list_shifted.insert(0, 0)

        # differences between the years list
        differences = [municipio_data_list_raw[i] - municipio_data_list_shifted[i] for i in range(len(municipio_data_list_raw))]
        # remove the first element of the list
        differences.pop(0)
        # find the index of the year with the biggest difference
        biggest_difference_index = differences.index(min(differences))
        # map the index to the year
        municipios_validos[municipio] = years2index[biggest_difference_index+1]
        # print(municipio_row_index, municipio)
        # print()
        # print(municipio_data_list_raw)
        # print(municipio_data_list_shifted)
        # print(differences)
        # print(biggest_difference_index)
        # print(years2index[biggest_difference_index+1])
        # print()
    
    return municipios_validos

perdaGrandesMunicipios()


In [12]:
def demografiaMunicipios() -> dict[str,tuple[str,str]]:
    # Por cada região NUTS III, qual o município que mais perdeu e o que mais ganhou eleitores entre 1975 e 2022? Complete a definição da função demografiaMunicipios, que retorna um dicionário { regiao : (municipioPerdeu,municipioGanhou) }.

    location_type = 'NUTS III'
    nuts_rows_df = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]
    #filtar para somente o Total
    nuts_rows = nuts_rows_df["Total"]

    location_type = 'Município'
    municipal_rows = legislativas[legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]


    nuts_municipality = {}
    # { regiao : [] }
    # exemplo: { 'Norte' : [4, 5, 6, 7, 8, 9, 10, 11, 12, 13] }

    for nuts_row_index in range(len(nuts_rows)):
        # list of row indexes that are under the NUTS III category
        # get the first row of the nuts_rows
        # print(nuts_rows.index[nuts_row_index])
        # check if the nuts_row_index is the last element of the list
        if nuts_row_index == len(nuts_rows)-1:
            list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, len(legislativas)))
        else:
            list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, nuts_rows.index[nuts_row_index+1]))
        # print(list_of_nuts)
        # add this nuts to the dictionary
        nuts_municipality[nuts_rows_df[('Territórios', 'Região')].iloc[nuts_row_index]] = list_of_nuts


    # --- ok --

    # pra cada municipio dentro de cada regiao, é necessario achar o valor da diferenca entre os anos 1975 e 2022 na coluna Total

    municipality_differences = {}
    municipality_data_dict = {}

    results = {}

    print(municipal_rows.head())

    print(municipal_rows.columns[0])

    municipal_rows.set_index(legislativas.columns[0], inplace=True)
    display(municipal_rows)
    # i whant to set the first column as the index
    # municipal_rows.set_index(('Territórios', 'Região'), inplace=True)

    # 1. acessar a primeira regiao
    for region in nuts_municipality:
        print("-------------------")
        # 2. acessar o primeiro municipio
        for municipality_index in nuts_municipality[region]:
            # 3. acessar a serie do municipio
            print(municipality_index, municipal_rows[('Territórios', 'Região')].iloc[municipality_index])
            municipality_data = legislativas["Total"].iloc[municipality_index]
            # 4. transformar a serie em uma lista
            municipality_data_list = municipality_data.tolist()
            # 5. achar a diferenca entre os anos 1975 e 2022
            difference = municipality_data_list[-1] - municipality_data_list[0]
            municipality_data_dict.update({legislativas[('Territórios', 'Região')].iloc[municipality_index]: difference})
        municipality_differences[region] = municipality_data_dict
        municipality_data_dict = {}
    
    for region in municipality_differences:
        print(region)
        print(municipality_differences[region])
        print()
        results[region] = (min(municipality_differences[region], key=municipality_differences[region].get), 
                           max(municipality_differences[region], key=municipality_differences[region].get))


    display(results)
    
    return None

demografiaMunicipios()

        Territórios                     Total                              \
  Âmbito Geográfico             Região   1975   1976   1979   1980   1983   
4         Município  Arcos de Valdevez  20217  20825  20745  21559  22136   
5         Município            Caminha   9942  10557  11323  11755  12364   
6         Município            Melgaço   8878   9140   9828  10041  10058   
7         Município             Monção  15453  16158  17045  17385  17927   
8         Município   Paredes de Coura   7577   7811   7973   8156   8351   

                        ... Votantes                                     \
    1985   1987   1991  ...     1991   1995   1999   2002   2005   2009   
4  23618  23959  25374  ...    13647  13911  13367  13343  12951  12519   
5  13073  13155  13825  ...     9704  10425   9564   9739   9995   9851   
6  10555  10753  11062  ...     5671   5891   5226   5174   5018   4950   
7  18806  19296  20095  ...    11653  11766  10386  10675  11003  10334   
8   8869  

Territórios  Total                \
                                             Região   1975   1976   1979   
(Territórios, Âmbito Geográfico)                                           
Município                         Arcos de Valdevez  20217  20825  20745   
Município                                   Caminha   9942  10557  11323   
Município                                   Melgaço   8878   9140   9828   
Município                                    Monção  15453  16158  17045   
Município                          Paredes de Coura   7577   7811   7973   
...                                             ...    ...    ...    ...   
Município                             Ribeira Brava   8018   8002   7858   
Município                                Santa Cruz  13459  13588  14131   
Município                                   Santana   6518   6760   6727   
Município                               São Vicente   4748   4848   4946   
Município                               Porto Santo   2369   2479   2537   

                                                                            \
                                   1980   1983   1985   1987   1991   1995   
(Territórios, Âmbito Geográfico)                                             
Município                         21559  22136  23618  23959  25374  26393   
Município                         11755  12364  13073  13155  13825  14543   
Município                         10041  10058  10555  10753  11062  11264   
Município                         17385  17927  18806  19296  20095  20524   
Município                          8156   8351   8869   9032   9367   9650   
...                                 ...    ...    ...    ...    ...    ...   
Município                          7948   8282   8824   9121  10198  10899   
Município                         14479  15187  16480  16782  18713  20368   
Município                          6891   7424   7505   8197   8226   8521   
Município                          5011   5236   5601   5623   5918   5930   
Município                          2638   2759   2990   3058   3424   3741   

                                  ... Votantes                              \
                                  ...     1991   1995   1999   2002   2005   
(Territórios, Âmbito Geográfico)  ...                                        
Município                         ...    13647  13911  13367  13343  12951   
Município                         ...     9704  10425   9564   9739   9995   
Município                         ...     5671   5891   5226   5174   5018   
Município                         ...    11653  11766  10386  10675  11003   
Município                         ...     5097   5454   5120   5098   5113   
...                               ...      ...    ...    ...    ...    ...   
Município                         ...     6611   6830   6383   6562   7170   
Município                         ...    12778  13971  13639  14582  18629   
Município                         ...     5270   5322   4864   4977   5315   
Município                         ...     3319   3444   3210   3324   3553   
Município                         ...     2551   2786   2548   2464   2751   

                                                                     
                                   2009   2011   2015   2019   2022  
(Territórios, Âmbito Geográfico)                                     
Município                         12519  11947  11926  11122  11262  
Município                          9851   9401   9113   8734   9066  
Município                          4950   4546   3846   3556   3678  
Município                         10334   9901   9401   8549   8855  
Município                          5313   4717   4376   4246   4457  
...                                 ...    ...    ...    ...    ...  
Município                          7090   7268   6513   6711   6421  
Município                         20295  21126  19838  20899  21047  
Município          

-------------------
4 Paredes de Coura
5 Ponte da Barca
6 Ponte de Lima
7 Valença
8 Viana do Castelo
9 Vila Nova de Cerveira
10 Amares
11 Barcelos
12 Braga
13 Esposende
-------------------
15 Vila Verde
16 Cabeceiras de Basto
17 Fafe
18 Guimarães
19 Mondim de Basto
20 Póvoa de Lanhoso
-------------------
22 Vila Nova de Famalicão
23 Vizela
24 Arouca
25 Espinho
26 Gondomar
27 Maia
28 Matosinhos
29 Oliveira de Azeméis
-------------------
31 Porto
32 Póvoa de Varzim
33 Santa Maria da Feira
34 Santo Tirso
35 São João da Madeira
36 Trofa
37 Vale de Cambra
38 Valongo
39 Vila do Conde
40 Vila Nova de Gaia
41 Boticas
42 Chaves
43 Montalegre
44 Ribeira de Pena
45 Valpaços
46 Vila Pouca de Aguiar
47 Amarante
-------------------
49 Castelo de Paiva
50 Celorico de Basto
51 Cinfães
52 Felgueiras
53 Lousada
54 Marco de Canaveses
-------------------
56 Penafiel
57 Resende
58 Alijó
59 Armamar
60 Carrazeda de Ansiães
61 Freixo de Espada à Cinta
62 Lamego
63 Mesão Frio
64 Moimenta da Beira
65 Murça
66 P

IndexError: single positional indexer is out-of-bounds

In [76]:
legislativas = pd.read_excel("dados/legislativas.xlsx",header=[0,1],sheet_name="Quadro")

def demografiaMunicipios() -> dict[str,tuple[str,str]]:
# Por cada região NUTS III, qual o município que mais perdeu e o que mais ganhou eleitores entre 1975 e 2022? Complete a definição da função demografiaMunicipios, que retorna um dicionário { regiao : (municipioPerdeu,municipioGanhou) }.

    local_legislativas = legislativas.copy()

    location_type = 'NUTS III'
    nuts_rows_df = local_legislativas[local_legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]
    #filtar para somente o Total
    nuts_rows = nuts_rows_df["Total"]

    location_type = 'Município'
    municipal_rows = local_legislativas[local_legislativas[('Territórios', 'Âmbito Geográfico')] == location_type]

    nuts_municipality = {}
    # { regiao : [] }
    # exemplo: { 'Norte' : [4, 5, 6, 7, 8, 9, 10, 11, 12, 13] }

    print(nuts_rows.index)
    print(municipal_rows.index)

 
    # Convert lists to sets
    first_set = set(nuts_rows.index)
    second_set = set(municipal_rows.index)
    
    # Determine the bounds
    min_first = min(first_set)
    max_first = max(first_set)
    
    # Generate the set of all possible numbers in the range that are not in the first set
    possible_fits = set(range(min_first, max_first + 1)) - first_set
    
    # All numbers in the second set that are not in the first set
    non_first_numbers = second_set - first_set
    
    # Sort the non-first numbers to prepare for grouping
    valid_numbers = sorted(non_first_numbers)
    
    # Group consecutive numbers
    grouped_numbers = []
    if valid_numbers:
        current_group = [valid_numbers[0]]
        
        for number in valid_numbers[1:]:
            if number == current_group[-1] + 1:
                current_group.append(number)
            else:
                grouped_numbers.append(current_group)
                current_group = [number]
        
        grouped_numbers.append(current_group)  # Add the last group

    for nuts_row_index, current_group in zip(range(len(nuts_rows)), enumerate(grouped_numbers)):
        nuts_municipality[nuts_rows_df[('Territórios', 'Região')].iloc[nuts_row_index]] = current_group[1]
    # for nuts_row_index in range(len(nuts_rows)):
    #     # list of row indexes that are under the NUTS III category
    #     # get the first row of the nuts_rows
    #     # print(nuts_rows.index[nuts_row_index])
    #     # check if the nuts_row_index is the last element of the list
    #     if nuts_row_index == len(nuts_rows)-1:
    #         list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, len(local_legislativas)))
    #     else:
    #         list_of_nuts = list(range(nuts_rows.index[nuts_row_index]+1, nuts_rows.index[nuts_row_index+1]))
    #     # print(list_of_nuts)
    #     # add this nuts to the dictionary
    #     nuts_municipality[nuts_rows_df[('Territórios', 'Região')].iloc[nuts_row_index]] = list_of_nuts


    # --- ok --

    # pra cada municipio dentro de cada regiao, é necessario achar o valor da diferenca entre os anos 1975 e 2022 na coluna Total

    print(nuts_municipality)

    municipality_differences = {}
    municipality_data_dict = {}

    results = {}

    municipal_rows.set_index(local_legislativas.columns[0], inplace=True)
    # i whant to set the first column as the index
    # municipal_rows.set_index(('Territórios', 'Região'), inplace=True)
    display(local_legislativas[('Territórios', 'Região')].iloc[292])
    display(local_legislativas[('Territórios', 'Região')].iloc[293])
    display(local_legislativas[('Territórios', 'Região')].iloc[294])
    display(local_legislativas[('Territórios', 'Região')].iloc[295])

    # 1. acessar a primeira regiao
    for region in nuts_municipality:
        print("-------------------")
        # 2. acessar o primeiro municipio
        for municipality_index in nuts_municipality[region]:
            # 3. acessar a serie do municipio
            print(municipality_index, local_legislativas[('Territórios', 'Região')].iloc[municipality_index])
            municipality_data = local_legislativas["Total"].iloc[municipality_index]
            # 4. transformar a serie em uma lista
            municipality_data_list = municipality_data.tolist()
            # 5. achar a diferenca entre os anos 1975 e 2022
            difference = municipality_data_list[-1] - municipality_data_list[0]
            municipality_data_dict.update({local_legislativas[('Territórios', 'Região')].iloc[municipality_index]: difference})
        municipality_differences[region] = municipality_data_dict
        municipality_data_dict = {}
    
    for region in municipality_differences:
        print(region)
        print(municipality_differences[region])
        print()
        results[region] = (min(municipality_differences[region], key=municipality_differences[region].get), 
                           max(municipality_differences[region], key=municipality_differences[region].get))
    
    return results

demografiaMunicipios()

Index([  3,  14,  21,  30,  48,  55,  67,  87,  98, 110, 130, 141, 156, 165,
       182, 195, 207, 220, 231, 242, 248, 262, 278, 294, 313, 315, 322, 325,
       327, 330, 334, 336, 339, 343, 354],
      dtype='int64')
Index([  4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
       ...
       345, 346, 347, 348, 349, 350, 351, 352, 353, 355],
      dtype='int64', length=308)
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 

'Vila Viçosa'

'Algarve'

'Algarve'

'Albufeira'

-------------------
4 Arcos de Valdevez
5 Caminha
6 Melgaço
7 Monção
8 Paredes de Coura
9 Ponte da Barca
10 Ponte de Lima
11 Valença
12 Viana do Castelo
13 Vila Nova de Cerveira
-------------------
15 Amares
16 Barcelos
17 Braga
18 Esposende
19 Terras de Bouro
20 Vila Verde
-------------------
22 Cabeceiras de Basto
23 Fafe
24 Guimarães
25 Mondim de Basto
26 Póvoa de Lanhoso
27 Vieira do Minho
28 Vila Nova de Famalicão
29 Vizela
-------------------
31 Arouca
32 Espinho
33 Gondomar
34 Maia
35 Matosinhos
36 Oliveira de Azeméis
37 Paredes
38 Porto
39 Póvoa de Varzim
40 Santa Maria da Feira
41 Santo Tirso
42 São João da Madeira
43 Trofa
44 Vale de Cambra
45 Valongo
46 Vila do Conde
47 Vila Nova de Gaia
-------------------
49 Boticas
50 Chaves
51 Montalegre
52 Ribeira de Pena
53 Valpaços
54 Vila Pouca de Aguiar
-------------------
56 Amarante
57 Baião
58 Castelo de Paiva
59 Celorico de Basto
60 Cinfães
61 Felgueiras
62 Lousada
63 Marco de Canaveses
64 Paços de Ferreira
65 Penafiel
66 Resend

{'Alto Minho': ('Paredes de Coura', 'Viana do Castelo'),
 'Cávado': ('Terras de Bouro', 'Braga'),
 'Ave': ('Vieira do Minho', 'Guimarães'),
 'Área Metropolitana do Porto': ('Porto', 'Vila Nova de Gaia'),
 'Alto Tâmega e Barroso': ('Montalegre', 'Chaves'),
 'Tâmega e Sousa': ('Cinfães', 'Penafiel'),
 'Douro': ('Torre de Moncorvo', 'Vila Real'),
 'Terras de Trás-os-Montes': ('Vinhais', 'Bragança'),
 'Região de Aveiro': ('Sever do Vouga', 'Aveiro'),
 'Região de Coimbra': ('Pampilhosa da Serra', 'Coimbra'),
 'Região de Leiria': ('Alvaiázere', 'Leiria'),
 'Viseu Dão Lafões': ('Vouzela', 'Viseu'),
 'Beira Baixa': ('Idanha-a-Nova', 'Castelo Branco'),
 'Beiras e Serra da Estrela': ('Sabugal', 'Guarda'),
 'Oeste': ('Bombarral', 'Torres Vedras'),
 'Médio Tejo': ('Mação', 'Ourém'),
 'Lezíria do Tejo': ('Coruche', 'Benavente'),
 'Grande Lisboa': ('Lisboa', 'Sintra'),
 'Península de Setúbal': ('Alcochete', 'Seixal'),
 'Alentejo Litoral': ('Odemira', 'Sines'),
 'Baixo Alentejo': ('Mértola', 'Beja'),

In [77]:
def find_differences(set1, set2):
    # Compute the symmetric difference between the two sets
    difference = set1.symmetric_difference(set2)
    return difference

xs = {'Alentejo Central': ('Estremoz', 'Évora'), 'Alentejo Litoral': ('Odemira', 'Sines'), 'Algarve': ('Monchique', 'Loulé'), 'Alto Alentejo': ('Nisa', 'Campo Maior'), 'Alto Minho': ('Paredes de Coura', 'Viana do Castelo'), 'Alto Tâmega e Barroso': ('Montalegre', 'Chaves'), 'Ave': ('Vieira do Minho', 'Guimarães'), 'Baixo Alentejo': ('Mértola', 'Beja'), 'Beira Baixa': ('Idanha-a-Nova', 'Castelo Branco'), 'Beiras e Serra da Estrela': ('Sabugal', 'Guarda'), 'Cávado': ('Terras de Bouro', 'Braga'), 'Douro': ('Torre de Moncorvo', 'Vila Real'), 'Grande Lisboa': ('Lisboa', 'Sintra'), 'Ilha Graciosa': ('Santa Cruz da Graciosa', 'Santa Cruz da Graciosa'), 'Ilha Terceira': ('Vila da Praia da Vitória', 'Angra do Heroísmo'), 'Ilha da Madeira': ('Porto Moniz', 'Funchal'), 'Ilha das Flores': ('Lajes das Flores', 'Santa Cruz das Flores'), 'Ilha de Porto Santo': ('Porto Santo', 'Porto Santo'), 'Ilha de Santa Maria': ('Vila do Porto', 'Vila do Porto'), 'Ilha de São Jorge': ('Calheta [R.A.A.]', 'Velas'), 'Ilha de São Miguel': ('Nordeste', 'Ponta Delgada'), 'Ilha do Corvo': ('Corvo', 'Corvo'), 'Ilha do Faial': ('Horta', 'Horta'), 'Ilha do Pico': ('Lajes do Pico', 'Madalena'), 'Lezíria do Tejo': ('Coruche', 'Benavente'), 'Médio Tejo': ('Mação', 'Ourém'), 'Oeste': ('Bombarral', 'Torres Vedras'), 'Península de Setúbal': ('Alcochete', 'Seixal'), 'Região de Aveiro': ('Sever do Vouga', 'Aveiro'), 'Região de Coimbra': ('Pampilhosa da Serra', 'Coimbra'), 'Região de Leiria': ('Alvaiázere', 'Leiria'), 'Terras de Trás-os-Montes': ('Vinhais', 'Bragança'), 'Tâmega e Sousa': ('Cinfães', 'Penafiel'), 'Viseu Dão Lafões': ('Vouzela', 'Viseu'), 'Área Metropolitana do Porto': ('Porto', 'Vila Nova de Gaia')}

find_differences(set(xs), demografiaMunicipios())


Index([  3,  14,  21,  30,  48,  55,  67,  87,  98, 110, 130, 141, 156, 165,
       182, 195, 207, 220, 231, 242, 248, 262, 278, 294, 313, 315, 322, 325,
       327, 330, 334, 336, 339, 343, 354],
      dtype='int64')
Index([  4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
       ...
       345, 346, 347, 348, 349, 350, 351, 352, 353, 355],
      dtype='int64', length=308)
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 

'Vila Viçosa'

'Algarve'

'Algarve'

'Albufeira'

-------------------
4 Arcos de Valdevez
5 Caminha
6 Melgaço
7 Monção
8 Paredes de Coura
9 Ponte da Barca
10 Ponte de Lima
11 Valença
12 Viana do Castelo
13 Vila Nova de Cerveira
-------------------
15 Amares
16 Barcelos
17 Braga
18 Esposende
19 Terras de Bouro
20 Vila Verde
-------------------
22 Cabeceiras de Basto
23 Fafe
24 Guimarães
25 Mondim de Basto
26 Póvoa de Lanhoso
27 Vieira do Minho
28 Vila Nova de Famalicão
29 Vizela
-------------------
31 Arouca
32 Espinho
33 Gondomar
34 Maia
35 Matosinhos
36 Oliveira de Azeméis
37 Paredes
38 Porto
39 Póvoa de Varzim
40 Santa Maria da Feira
41 Santo Tirso
42 São João da Madeira
43 Trofa
44 Vale de Cambra
45 Valongo
46 Vila do Conde
47 Vila Nova de Gaia
-------------------
49 Boticas
50 Chaves
51 Montalegre
52 Ribeira de Pena
53 Valpaços
54 Vila Pouca de Aguiar
-------------------
56 Amarante
57 Baião
58 Castelo de Paiva
59 Celorico de Basto
60 Cinfães
61 Felgueiras
62 Lousada
63 Marco de Canaveses
64 Paços de Ferreira
65 Penafiel
66 Resend

set()

# Tarefa 4
---

In [95]:
nominations = pd.read_csv("dados/nominations.csv")

ramon_rows = nominations[nominations["Nominee(s)"].str.contains("Ramón Menéndez Pidal")]
display(ramon_rows)

# store the ramon_rows in a csv file
ramon_rows.to_csv("ramon_rows.csv")

unique_nominators = set(ramon_rows["Nominator(s)"].unique())
#print(unique_nominators)

# split the nominators based on the \r\n character and create a new line for each split
split_entries = ramon_rows["Nominator(s)"].str.split("\r\n").explode().unique()
# remove the white spaces from the split_entries
split_entries = [entry.strip() for entry in split_entries]

print(split_entries)
# find the duplicates in the split_entries
print("=====================================")
# remove duplicates from the data list
print(set(data))

# data.append("Larissa")

print(f"Raw: {len(split_entries)} ==  My: {len(set(data))}")

def find_duplicates(input_list):
    seen = set()
    duplicates = set()
    for item in input_list:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return list(duplicates)

# check witch entries are in the split_entries but not in the data
def difference(list1, list2):
    """
    This function returns the elements that are in list1 but not in list2.
    
    Args:
    list1 (list): The first list to compare.
    list2 (list): The second list to compare.

    Returns:
    list: A list containing elements that are in list1 but not in list2.
    """
    return [item for item in list1 if item not in list2]

print(find_duplicates(split_entries))
difference(set(data), split_entries)

,Year,Category,Nominee(s),Nominator(s)
12821,1931,Literature,Ramón Menéndez Pidal,49 members of The Nobel Prize Committee of the...
12822,1931,Literature,Ramón Menéndez Pidal,52 representatives
12823,1931,Literature,Ramón Menéndez Pidal,7 members of Faculté de Philosophie et Lettres...
12824,1931,Literature,Ramón Menéndez Pidal,14 members of the Portuguese Academy of Science
12825,1931,Literature,Ramón Menéndez Pidal,6 members of Real Academia de Bellas Artes de ...
...,...,...,...,...
14778,1965,Literature,Ramón Menéndez Pidal,O Deutschmann
14883,1966,Literature,Ramón Menéndez Pidal,Gunnar Tilander
14884,1966,Literature,Ramón Menéndez Pidal,Henri Guiter
14987,1967,Literature,Ramón Menéndez Pidal,Gunnar Tilander


['49 members of The Nobel Prize Committee of the Society of Authors', '52 representatives', '7 members of Faculté de Philosophie et Lettres, Brussels;', '14 members of the Portuguese Academy of Science', '6 members of Real Academia de Bellas Artes de Toledo', '21 members of the Royal Academy of History', 'Santiago Ramon y Cajàl', '27 members of the Royal Spanish Academy (Real Academia Española )', 'Erik Staaff', '30 members of the Royal Spanish Academy (Real Academia Española )', 'Per Hallström', 'Gunnar Tilander', 'Knut Hjalmar Leonard Hammarskjöld', 'Walther von Wartburg', 'Alonso Zamora Vicente', 'Chilean Academy of Language (Academia Chilena de la Lengua)', '100 institutions and individuals', 'Academie Argentina de Letras', 'Royal Spanish Academy', 'Rudolf Grossmann', 'Angel Rosenblat', '9 professors at the faculty of Humanities', 'José Ramon Ayala', 'Mariano Picon Salas', 'William Smith', 'Concha Zardoya', 'Raymond Willis', 'Norman Torrey', 'James Shearer', 'Vicente Gaos', 'Ernest

[]

In [96]:
import pandas as pd
import networkx as nx

# Creating a DataFrame from the provided data
nominations = pd.read_csv("dados/nominations.csv")
nominations = nominations.copy()


data = []

def maisNomeado():
    G = nx.DiGraph()

    # Percorrendo cada linha do DataFrame
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        year = row['Year']

        # Adicionando cada nomeado ao grafo
        for nominee in nominees:
            for nominator in nominators:
                if G.has_edge(nominator, nominee):
                    G[nominator][nominee]['years'].add(year)
                else:
                    G.add_edge(nominator, nominee, years={year})

    # Contagem dos nominadores únicos por nomeado
    nominee_counts = {}
    
    for nominator, nominee in G.edges():
        if nominee not in nominee_counts:
            nominee_counts[nominee] = set()
        nominee_counts[nominee].add(nominator)

    # Encontrar o nomeado com o maior número de nominadores únicos
    most_nominated = None
    max_nominators = 0
    for nominee, nominators in nominee_counts.items():
        if len(nominators) > max_nominators:
            most_nominated = nominee
            max_nominators = len(nominators)

    return most_nominated, max_nominators

maisNomeado()

('Ramón Menéndez Pidal', 147)

## 2
---

In [1]:
import pandas as pd
import networkx as nx
from collections import defaultdict

# Carrega os dados novamente
nominations = pd.read_csv("dados/nominations.csv")

def nomeacoesCruzadas():
    # Criar o grafo direcionado
    G = nx.DiGraph()

    # Adiciona as arestas baseadas nas nomeações
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        category = row["Category"]
        for nominator in nominators:
            for nominee in nominees:
                # Adiciona uma aresta do nominador para o nomeado
                G.add_edge(nominator, nominee, category=category)
    
    # Encontrar o maior componente fortemente conectado
    largest_scc = max(nx.strongly_connected_components(G), key=len)
    
    # Coletar categorias envolvidas neste componente
    categories = set()
    for u, v, data in G.edges(data=True):
        if u in largest_scc and v in largest_scc:
            categories.add(data['category'])

    # Retorna o tamanho do maior SCC e as categorias envolvidas
    return len(largest_scc), categories

# Chamada da função
print(nomeacoesCruzadas())

(348, {'Peace', 'Medicine', 'Chemistry', 'Physics'})


## 3
---


In [6]:
import pandas as pd
import networkx as nx

# Carregando os dados
nominations = pd.read_csv("dados/nominations.csv")

def caminhoEinsteinFeynman() -> list[str]:

    local_nominations = nominations.copy()
    
    # Filtrar dados para o período entre 1921 e 1965 e para a categoria de Física
    local_nominations = local_nominations[(local_nominations['Year'] >= 1921) & (local_nominations['Year'] <= 1965) & (local_nominations['Category'] == 'Physics')]
    
    # Criando o grafo
    G = nx.DiGraph()
    
    for idx, row in local_nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        
        # Adicionando arestas de cada nominador para cada nomeado
        for nominator in nominators:
            for nominee in nominees:
                G.add_edge(nominator, nominee)
    
    nx.write_graphml(G, "Albert2Feyman.graphml")

    # Encontrando o caminho entre Einstein e Feynman
    # Einstein como nominador (teremos que verificar se ele nomeou alguém diretamente)
    # Feynman como nomeado
    try:
        # Inicialmente assumimos que Einstein e Feynman são parte dos nodos
        # Este método levanta uma exceção se não houver caminho
        # Source and target nodes
        source = 'Albert Einstein'
        target = 'Richard Phillips Feynman'

        # Find all shortest paths
        all_shortest_paths = list(nx.all_shortest_paths(G, source=source, target=target))
        # Extract only intermediate nodes from each path
        all_shortest_paths = [path[1:-1] for path in all_shortest_paths]  # Slicing to exclude the first and last elements
        return all_shortest_paths
    except nx.NetworkXNoPath:
        return []  # Retornar lista vazia se não houver caminho


# Teste da função
caminhoEinsteinFeynman()

[['James Franck', 'Emilio Gino Segrè'],
 ['Walther Bothe', 'Emilio Gino Segrè'],
 ['Arthur Compton', 'Carl Anderson'],
 ['Werner Karl Heisenberg', 'Hans Albrecht Bethe'],
 ['Isidor Rabi', 'Hans Albrecht Bethe']]